# Computation of the contact function using the Cholesky decomposition of Q and a Newton-Raphson approach

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg
import scipy.optimize

In [ ]:
%matplotlib inline

In [ ]:
import pypw85

In [ ]:
def to_array_2d(a):
    return np.array([[a[0], a[1], a[2]],
                     [a[1], a[3], a[4]],
                     [a[2], a[4], a[5]]])

In [ ]:
golden_ratio = (1.+np.sqrt(5.))/2.
norm = np.sqrt(1+golden_ratio**2)
u_abs = 1./norm
v_abs = golden_ratio/norm

r = np.array([0., u_abs, v_abs])
n1 = np.array([0., u_abs, v_abs])
n2 = np.array([0., u_abs, v_abs])
a1 = c1 = 1.999
a2 = c2 = 0.0199

q1 = pypw85.spheroid(a1, c1, n1)
q2 = pypw85.spheroid(a2, c2, n2)

out = np.empty((2,), dtype=np.float64)
pypw85.contact_function(r, q1, q2, out)
μ2, λ = out

print('μ² = {} (actual value)'.format(μ2))
print('λ  = {} (actual value)'.format(λ))

Q1, Q2 = to_array_2d(q1), to_array_2d(q2)
Q = (1-λ)*Q1 + λ*Q2

y = np.linalg.solve(Q, r)
print('y = {}'.format(y))

c1_x0 = np.dot((1-λ)*Q1, y)
c2_x0 = -np.dot(λ*Q2, y)
r_actual = c1_x0-c2_x0
print(r_actual, r)

μ2_1 = np.linalg.solve(Q1, c1_x0).dot(c1_x0)
μ2_2 = np.linalg.solve(Q2, c2_x0).dot(c2_x0)

print('μ²  = {} (value found from the contact_function routine)'.format(μ2))
print('μ₁² = {} (first post-processed value)'.format(μ2_1))
print('μ₂² = {} (second post-processed value)'.format(μ2_2))

In [ ]:
def f(λ, r, Q1, Q2):
    Q = (1-λ)*Q1+λ*Q2
    L = scipy.linalg.cholesky(Q, lower=True)
    LTs = scipy.linalg.solve_triangular(L, r, lower=True)
    s = scipy.linalg.solve_triangular(L, LTs, lower=True, trans=1)
    return λ*(1-λ)*r.dot(s)

def f1(λ, r, Q1, Q2):
    Q = (1-λ)*Q1+λ*Q2
    print(Q)
    dQ = Q2-Q1
    L = scipy.linalg.cholesky(Q, lower=True)
    LTs = scipy.linalg.solve_triangular(L, r, lower=True)
    s = scipy.linalg.solve_triangular(L, LTs, lower=True, trans=1)
    return (1-2*λ)*r.dot(s)-λ*(1-λ)*s.dot(dQ.dot(s))

def f2(λ, r, Q1, Q2):
    Q = (1-λ)*Q1+λ*Q2
    Q_prime = Q2-Q1
    L = scipy.linalg.cholesky(Q, lower=True)
    LTs = scipy.linalg.solve_triangular(L, r, lower=True)
    s = scipy.linalg.solve_triangular(L, LTs, lower=True, trans=1)
    u = Q_prime.dot(s)
    LTv = scipy.linalg.solve_triangular(L, u, lower=True)
    v = scipy.linalg.solve_triangular(L, LTv, lower=True, trans=1)
    return -2*r.dot(s)-2*(1-2*λ)*s.dot(u)+2*λ*(1-λ)*u.dot(v)

In [ ]:
x = np.linspace(0., 1.0, num=1001)
y = np.array([f(x_i, r, Q1, Q2) for x_i in x])
y1 = np.array([f1(x_i, r, Q1, Q2) for x_i in x])

In [ ]:
plt.plot(x, y1)
plt.ylim((-0.3, 0.3))

In [ ]:
μ2_exp = (np.linalg.norm(r)/(a1+a2))**2

In [ ]:
μ2_exp

In [ ]:
x_opt = scipy.optimize.newton(lambda x: f1(x, r, Q1, Q2), 1.0, fprime=lambda x: f2(x, r, Q1, Q2))

In [ ]:
(f(x_opt, r, Q1, Q2)-μ2_exp)/μ2_exp